# Filter and Charts

Filters and Charts both take senders and receivers.  [Note:  chart sender is a work in progress.]

### Standalone Filters
A new feature added with Sonar is the ability to show() a Filter without adding it to a chart -- a "standalone filter."  Using a receiver on a chart, a chart can be filtered without actually adding a Filter to the JoogleChart.  A standalone filter can be used to send the filtering values, allowing you to essentially roll your own chart filter.  Or, other Jooglechart widgets can be used to send filtering values, such as CheckboxGroup or ButtonGroup.

What is the point of rolling your own filter?  Disconnecting the Filter from the chart allows much more flexibility in layout, particularly when used in combination with the layout containers -- ChartRow and Box.  In addition, one Filter can be used to modify two or more charts, and those objects can be more easily laid out in a sensible fashion.

# Filters

### .add_sender(key, on="statechange", message_type="default")

In most cases, you will only specify the key, and leave the other parameters on their default value.

#### key
An arbitrary string used to connect the sender to a receiver.  The receiver must get the same key.

#### on
Specifies the event that triggers the message.  Currently, the only supported event that triggers a message is "statechange."  The statechange event is simply making a selection on the filter.  (Google filters only offer three events:  error, ready, and statechange.  Only statechange is supported in Jooglechart.)
- statechange

#### message_type
Specifies the type of message that is triggered by the event.
- "default":  If the filter is a CategoryFilter, defaults to "values".  If it's a range filter, defaults to "range".
- "values":  a list of selected values.  Used with CategoryFilter.
- "range":  a list with minimum and maximum values.  Used with NumberRangeFilter and DateRangeFilter.

### .add_receiver(key, action="default" [, kwargs])

#### key
An arbitrary string used to connect the receiver to a sender.  The key must match that of the sender.

#### action
The action the filter will take when receiving the message.
- "default":  If the filter is a CategoryFilter, defaults to "update_values".  If it's a range filter, defaults to "update_range".
- "update_values":  Updates the selected values on the filter.
- "update_range":  Updates the range selection on the filter.
- "update_binding_values":  Use this action to create your own filter binding behavior.  It will "filter" the options in a CategoryFilter by the values in another column (the bound column).  You must pass the kwarg "bound_column" with an integer for the bound column.

# Charts


#### .add_sender(key, on="select", message_type="category" [, kwargs]) 

#### key
An arbitrary string used to connect the sender to a receiver.

#### on
Currently there is only supported event is "select."  The select event is the user clicking on the chart.

#### message_type
The type of message triggered by the event.
- "category" (default):  The category of the selection a user clicks on the chart.  You must pass the column of the category in a keyword argument.
- "header (not yet implemented):  The column header of the selection the user clicks on.
- "value" (not yet implemented):  The underlying data value of the selection the user clicks on.

### .add_receiver(key, action="filter_values" [, kwargs])

#### key
An arbitrary string to connect the sender with a receiver.

#### action
- "filter_values":  Filter the data rows by the list of values passed.  You must pass the kwarg "column" with an integer for the column to be filtered.
- "filter_range":  Filter the data rows by range (min and max) passed.  You must pass the kwarg "column" for the column to be filtered.
- "filter_columns":  Filter the data columns by list of values.  Use this to roll your own series filter.
- "focus_column":  Trigger a highlight of a data column, as if the user clicked on a legend key.  Use this with the Legend widget.  [Note:  I've tried to trigger the highlight style of a legend mouseover, but can't get it working.]


# Gallery



In [30]:
from jooglechart import JoogleChart, Filter, ChartRow, SeriesFilter
from jooglechart.utils import get_df, get_date_df
from jooglechart.widgets import Box, Button
import pandas as pd

### CategoryFilter Sender, CategoryFilter Receiver

In [3]:
df1 = get_df()
df2 = get_df(cols=3)

chart1 = JoogleChart(df1)
filter1 = Filter("CategoryFilter")
filter1.add_options(filterColumnIndex=0)
filter1.add_sender(key="APPLE")  # Add a sender to the first filter with a key
chart1.add_filter(filter1)
chart1.add_chart_options(legend_position="top", colors=['brown'])

chart2 = JoogleChart(df2)
filter2 = Filter("CategoryFilter")
filter2.add_options(filterColumnIndex=0)
filter2.add_receiver(key="APPLE")  # Add a receiver to the second filter, using the same key
chart2.add_filter(filter2)
chart2.add_chart_options(legend_position="top", colors=['SlateGray', 'Tomato', 'DarkOrchid'])

row = ChartRow(chart1, chart2)
row.show()

### NumberRangeFilter Sender, NumberRangeFilter Receiver

In [4]:
df = get_df(min=10, max=20)
chart1 = JoogleChart(df, chart_type="ColumnChart")
filter1 = Filter("NumberRangeFilter")
filter1.add_options(filterColumnIndex=1)
filter1.add_sender(key="STRAWBERRY")
chart1.add_filter(filter1)
chart1.add_chart_options(legend_position="top", colors=['burlywood'])

chart2 = JoogleChart(df, chart_type="BarChart")
filter2 = Filter("NumberRangeFilter")
filter2.add_receiver(key="STRAWBERRY")
filter2.add_options(filterColumnIndex=1)
chart2.add_filter(filter2)
chart2.add_chart_options(legend_position="top", colors=['cadetblue'])

row = ChartRow(chart1, chart2)
row.show()

### Standalone CategoryFilter, Chart Receiver

In [5]:
df = get_df()

f = Filter("CategoryFilter", data=df.cities)
f.add_sender(key="PLUM")
f.add_options(filterColumnIndex=0)

chart1 = JoogleChart(df)
chart1.add_receiver(key="PLUM", column=0)
chart1.add_chart_options(legend_position="top", colors=["MediumSeaGreen"])

df2 = get_df(cols=2)
chart2 = JoogleChart(df2, chart_type="BarChart")
chart2.add_receiver(key="PLUM", column=0)
chart2.add_chart_options(legend_position="top", colors=['Chocolate','DodgerBlue'])

row = ChartRow(chart1, chart2, mode="free")
box = Box(f, row)
box.show()

### Standalone DateRangeFilter, Chart Receiver

In [6]:
df = get_date_df(days=10)

f = Filter("DateRangeFilter", data=df.dates)
f.add_options(filterColumnIndex=0)
f.add_sender(key="CCC")

chart1 = JoogleChart(df, chart_type="LineChart")
chart1.add_receiver(key="CCC", column=0, action="filter_range")
chart1.add_chart_options(legend_position="top", width=600)

box = Box(f, chart1)
box.show()

### A custom Series filter:  filter_columns on a chart
When rolling you own series filter, you can use the chart's **.get_viewable_series()** method.  It will return a list of viewable columns, removing role columns and hidden columns.

In [7]:
df = get_df(cols=4, rows=3)
    
chart = JoogleChart(df)
chart.add_receiver(key="GRAPES", column=0, action="filter_columns")
chart.add_chart_options(legend_position="top")

series = chart.get_viewable_series()
f1 = Filter("CategoryFilter", data=pd.Series(series, name="Letters"))
f1.add_sender(key="GRAPES")
f1.add_options(filterColumnIndex=0)
f1.add_state(selectedValues=series[1:3])

row = ChartRow(f1, chart, mode="weighted", weights=[1, 3])
row.show()

### Binding filters
You can roll your own bound filters when there is more than category column slicing up the data.  With bound filters, one filter changes the options available in another filter.  But both of those filters will update the chart.  Be sure to connect the chart to all the filters.

In [8]:
df = get_df(category_column=True, rows=15)
f1 = Filter("CategoryFilter", data = df)
f1.add_options(filterColumnIndex = 1)
f1.add_sender(key="PEAR")  # both the bound filter and the chart must respond to PEAR

f2 = Filter("CategoryFilter", data = df)
f2.add_options(filterColumnIndex = 0)
f2.add_receiver(key="PEAR", action="update_binding_values", bound_column=1)
f2.add_sender(key="APRICOT")

chart = JoogleChart(df)
chart.add_chart_options(legend_position="top", colors=["FireBrick"], width="700", height=350)
chart.add_receiver(key="PEAR", column=1)
chart.add_receiver(key="APRICOT", column=0)
chart.set_view_cols([0, 2])

box = Box(f1, f2)
row = ChartRow(box, chart, mode="free", padding=20)

print df  # dataframe is displayed for demonstration purposes
row.show()

         cities categories  Aaaaaaaaa
0    Sacramento          C        100
1    Montpelier          C         96
2        Juneau          A         64
3    Montgomery          A         56
4   Little Rock          C         41
5       Phoenix          C         22
6        Denver          C         22
7      Hartford          A         57
8   Tallahassee          B         45
9         Dover          B         50
10      Atlanta          B         92
11     Honolulu          A         62
12       Austin          B         65
13       Albany          B         39
14    Frankfurt          C         21


### Chart sender:  Click to send category

In [66]:
df = get_df(category_column=True, rows=15)

df_agg = df.groupby(df['categories'], as_index=False).sum()
df_agg.reset_index()

chart1 = JoogleChart(df_agg)
chart1.add_chart_options(legend_position="top")
chart1.add_chart_options(width=400, chartArea_left=50, chartArea_width=450)
chart1.add_sender("RASPBERRY", column=0)

chart2 = JoogleChart(df, chart_type="Table")
chart2.add_receiver("RASPBERRY", column=1)

clear = Button(value=None, text="clear filter")
clear.add_sender("RASPBERRY")
clear.add_div_styles({'margin-top': "10px"})

title_row = ChartRow("<h4>Click the column to filter the table</h4>", clear, mode="free", padding=10)
row = ChartRow(chart1, chart2, mode="fixed", widths=[500, 400])
box = Box(title_row, row)
box.show()